In [39]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request
import re     

import pandas as pd
import numpy as np

import time
from tqdm.notebook import tqdm

time_sleep_second = 0.5

In [55]:
cat2 = {'전체':'', '일상':12,'초스피드':18,'손님접대':13,'술안주':19,'다이어트':21,'도시락':15,'영양식':43,'간식':17,'야식':45,'푸드스타일링':20,
       '해장':46, '명절':44, '이유식':14, '기타':2}

reverse_cat2 = {12: '일상', 18: '초스피드', 13: '손님접대', 19: '술안주', 21: '다이어트',
 15: '도시락', 43: '영양식', 17: '간식', 45: '야식', 20: '푸드스타일링', 46: '해장', 44: '명절', 14: '이유식', 2: '기타'}

category_number_list = list(reverse_cat2.keys())
category_number_list

[12, 18, 13, 19, 21, 15, 43, 17, 45, 20, 46, 44, 14, 2]

In [58]:
recipe_final_list = [] #여따가 최종 결과물 담을거임

for category_num in category_number_list: #위에서 정의한 카테고리별 number로 카테고리별 페이지 접근
    category = reverse_cat2[category_num]
    category_first_page = f'https://www.10000recipe.com/recipe/list.html?cat2={category_num}&order=reco&page=1'

    r_category = requests.get(category_first_page)
    soup_category = BeautifulSoup(r_category.content, "html.parser")

    num_of_recipe_in_category = int(soup_category.select('#contents_area_full > div.rcp_m_list2 > div.m_list_tit > b')[0].text.replace(',', '')) #카테고리 선택 하단에 노출되는 해당 카테고리의 레시피 개수 가져오기
    page_view = num_of_recipe_in_category // 44 #카테고리별 페이지의 1페이지당 노출 박스가 44개고(광고 3개), 위에서 구한 카테고리별 총 레시피 개수를 이로 나누어 주면 총 페이지수 짐작이 가능함. 44 - 3 = 41로 하면 정확한 페이지 구할 수 있을 것 같지만 시도해보지는 않음

#     if page_view >= 500:
#         page_view = 500
## 페이지 개수 과도하게 많아지면 그냥 500개 정도에서 끊겠다는 코드

    for page in range(1, page_view+1): #카테고리별 페이지의 1페이지 당 노출되는 recipe들의 id list를 크롤링
        category_page = f'https://www.10000recipe.com/recipe/list.html?cat2={12}&order=reco&page={page}'
        r_page = requests.get(category_page)
        soup_page=BeautifulSoup(r_big.content, "html.parser")

        recipe_id_list = []
        food_list = soup_page.select('#contents_area_full > div.rcp_m_list2 > div.row')[0].find_all('div')

        for food in food_list:
            try:
                id_num = food.find('a', {'class':'thumbnail'}).attrs['href'].split('/')[-1]
                recipe_id_list.append(id_num)
            except:
                pass

        recipe_id_list = list(set(recipe_id_list)) 

        for id_num in recipe_id_list: #위에서 구한 id로 레시피 별 url에 접근해서 크롤링

            url = f'https://www.10000recipe.com/recipe/{id_num}'
            r = requests.get(url)
            soup=BeautifulSoup(r.content, "html.parser")

            basic_info = soup.select('#contents_area > div.view2_summary > div.view2_summary_info')[0].find_all('span')
            ingredients = soup.select('#divConfirmedMaterialArea > ul')


            title = soup.find('title').text
            try:
                amount = basic_info[0].text
            except:
                amount = 'None'
            try:
                duration = basic_info[1].text
            except:
                duration = 'None'
            try:
                level = basic_info[2].text
            except:
                level = 'None'

            temp_material = []
            temp_seasoning = []

            try:
                ingredient_string = ''

                for ingredient in ingredients:
                    for material in ingredient.find_all('li'):
                        temp_material.append(material.text.replace('\n', " ").replace(" ",""))

                temp_string = ', '.join(temp_material)
                ingredient_string = ingredient_string + temp_string +  ', '

            except:
                ingredient_string = ''

            row = [title, amount, duration, level, ingredient_string, id_num, url, category]
            recipe_final_list.append(row)

            time.sleep(time_sleep_second)

In [60]:
crawled = pd.DataFrame(recipe_final_list, columns = ['레시피', '양', '소요시간', '난이도', '재료', '레시피ID', 'URL', '카테고리'])
crawled

,레시피,양,소요시간,난이도,재료,레시피ID,URL,카테고리
0,애호박 덮밥 혼밥으로 딱 좋아~,2인분,15분 이내,아무나,"애호박1개, 양파1/2개, 굴소스1큰술, 고춧가루1큰술, 다진마늘1t, 참기름, 식...",6881286,https://www.10000recipe.com/recipe/6881286,일상
1,"[원팬,원볼] 게다가 노버터 인데 촉촉한 모카 컵케이크?! 가능해요!",5인분,30분 이내,아무나,"박력분140g, 설탕100g, 소금1g, 베이킹파우더2g, 베이킹소다1g, 계란1개...",6930570,https://www.10000recipe.com/recipe/6930570,일상
2,"배추 된장 무침, 간단 달달한 배추 고추장무침",4인분,10분 이내,아무나,"삶은배추3덩어리(+봄동가능), 된장1큰술, 고추장1/2큰술, 참기름1/2큰술, 다진...",6904476,https://www.10000recipe.com/recipe/6904476,일상
3,애호박 덮밥 혼밥으로 딱 좋아~,2인분,15분 이내,아무나,"애호박1개, 양파1/2개, 굴소스1큰술, 고춧가루1큰술, 다진마늘1t, 참기름, 식...",6881286,https://www.10000recipe.com/recipe/6881286,일상
4,"[원팬,원볼] 게다가 노버터 인데 촉촉한 모카 컵케이크?! 가능해요!",5인분,30분 이내,아무나,"박력분140g, 설탕100g, 소금1g, 베이킹파우더2g, 베이킹소다1g, 계란1개...",6930570,https://www.10000recipe.com/recipe/6930570,일상
5,"배추 된장 무침, 간단 달달한 배추 고추장무침",4인분,10분 이내,아무나,"삶은배추3덩어리(+봄동가능), 된장1큰술, 고추장1/2큰술, 참기름1/2큰술, 다진...",6904476,https://www.10000recipe.com/recipe/6904476,일상
6,애호박 덮밥 혼밥으로 딱 좋아~,2인분,15분 이내,아무나,"애호박1개, 양파1/2개, 굴소스1큰술, 고춧가루1큰술, 다진마늘1t, 참기름, 식...",6881286,https://www.10000recipe.com/recipe/6881286,일상
7,"[원팬,원볼] 게다가 노버터 인데 촉촉한 모카 컵케이크?! 가능해요!",5인분,30분 이내,아무나,"박력분140g, 설탕100g, 소금1g, 베이킹파우더2g, 베이킹소다1g, 계란1개...",6930570,https://www.10000recipe.com/recipe/6930570,일상
8,"배추 된장 무침, 간단 달달한 배추 고추장무침",4인분,10분 이내,아무나,"삶은배추3덩어리(+봄동가능), 된장1큰술, 고추장1/2큰술, 참기름1/2큰술, 다진...",6904476,https://www.10000recipe.com/recipe/6904476,일상
